https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html



Uncommend and run the following pip & python commands when running a new compute for the <b> first</b> time! 

In [491]:
#!python -m spacy download el_core_news_sm
#!pip install pyarrow --upgrade
#!pip install openpyxl
#!pip install xlrd
#!pip install spacy

In [492]:
import spacy
import el_core_news_sm
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [493]:
from spacy.lang.el import GreekLemmatizer

In [494]:

#lemmatizer = GreekLemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [495]:
nlp =spacy.load('el_core_news_sm')

In [496]:

p1 = re.compile('δεν απαντ.{1,3}\s{0,1}',re.IGNORECASE)
p2 = re.compile('\sδα\s',re.IGNORECASE)
p3 = re.compile('δε.{0,1}\s.{0,3}\s{0,1}βρ.{1,2}κ.\s{0,1}',re.IGNORECASE)
p4 = re.compile('[^\d]?\d{10}')
p5 = re.compile('[^\d]?\d{18}|[^\d]\d{20}')
p6 = re.compile('δε[ ν]{0,1} (επιθυμ[α-ω]{2,4}?|[ήη]θ[εέ]λ[α-ω]{1,3}?|θελ[α-ω]{1,4}|.{0,20}ενδιαφ[εέ]ρ[α-ω]{2,4})',re.IGNORECASE)
p7 = re.compile('δε[ ν]{0,1} (μπορ[α-ω]{2,5}|.εχει)',re.IGNORECASE)
p8 = re.compile('(δεν|μη).*διαθεσιμ[οη]ς{0,1}?',re.IGNORECASE)
p9 = re.compile('(δεν|μη)+.*εφικτη?',re.IGNORECASE)
p10 = re.compile('δε[ ν]{0,1}.{0,20}θετικ[οόήη]ς{0,1}',re.IGNORECASE)

In [497]:
def loadStopWords():
    sWords = open('stopWords.txt','r',encoding='utf-8')
    sw = set(sWords.read().split('\n'))
    #sw = sw.remove('μη')
    sWords.close()
    return sw

In [498]:
def replaceTerm(text):
    text = p5.sub(' λογαριασμός ',text)
    text = p4.sub(' τηλεφωνο ',text)
    text = p6.sub(' δενθελειδενενδιαφερεται ',text)
    text = p10.sub(' δενθελειδενενδιαφερεται ',text)
    text = p7.sub(' δενεχειδενμπορει ',text)
    text = p8.sub(' δενειναιδιαθεσιμος ',text)
    text = p9.sub(' ανεφικτη ',text)
    text = text.replace('-banking','banking')
    text = text.replace('v banking','vbanking')
    text = text.replace('e banking','ebanking')
    text = text.replace('follow up','followup')
    text = text.replace('fup','followup')
    text = text.replace('f/up','followup')
    text = text.replace('πυρ/ριο','πυρασφαλιστηριο')
    text = text.replace('safe drive','safedrive')
    text = text.replace('safe pocket','safepocket')
    text = text.replace('alphabank','alpha')
    text = text.replace('sweet home smart','sweethomesmart')
    text = text.replace('sweet home','sweethome')
    text = text.replace('eξασφαλιζω','εξασφαλιζω')
    text = text.replace('credit card','creditcard')
    text = text.replace('debit card','debitcard')
    text = text.replace('life cycle','lifecycle')
    text = text.replace('π/κ','πκ')
    text = text.replace('td','πκ')
    text = text.replace('α/κ','ακ')
    text = text.replace('δ/α','δεναπαντα ')
    text = text.replace('εκτος αττικης','εκτοςαττικης ')
    #τδ
    text = p1.sub(' δεναπαντα ',text)
    text = p2.sub(' δεναπαντα ',text)
    text = p3.sub(' δεντονβρηκα ',text)
    
    return text


In [499]:


#sw = nlp.Defaults.stop_words
#sw = sw|{'εχω','απο','ωστε'}
sw = loadStopWords()
def remove_ton(text):
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   
def clean_text(text):
     #text to string
    text = str(text).lower()
    text = replaceTerm(text)
    
   # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # lower text
    text = [remove_ton(x) for x in text]
    # remove stop words
    text = [x for x in text if x not in sw]
 
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in {'quot','amp'}]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 1])
     # lemmatize text
    text = " ".join([str(t) for t in nlp(text)])
   
    return(text)

In [500]:
def correct(x,corDict):
    if x in corDict.keys():
        y = corDict[x]
    else:
        y = x
    return y    

In [501]:
def get_ngrams(idf,mindf,minngram,maxngram):
    tfidf = TfidfVectorizer(min_df = mindf,ngram_range = (minngram,maxngram))
    tfidf_result = tfidf.fit_transform(idf['tokenized']).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = [str(x) for x in tfidf_df.columns]
    df_i = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()
    df_i = df_i[df_i['value']>0]
    return df_i

In [502]:
def cleanComments(df):
    df = df[['CON_ROW_ID','CON_COMMENTS']]
    df['tokenized'] = df['CON_COMMENTS'].apply(clean_text)
    df = df.fillna('N/A')
    df['variable'] = df['tokenized'].str.split()
    return df

In [503]:
def getTokens(df):
    df = cleanComments(df)
    df_f = df.explode('variable')[['CON_ROW_ID','variable']]
    return df_f


In [504]:
def getTokencount(df_f,minCount):
    tokenCount = df_f['variable'].value_counts().to_dict()
    df_f['value'] = df_f['variable'].map(tokenCount)
    df_f=df_f[df_f['value']>minCount] 
    return df_f

In [505]:
txt = 'Τον ενημέρωσα για τη καμπάνια bancassurance και δεν ενδιαφέρεται'

In [506]:
[t for t in nlp(txt, disable=['tagger', 'parser', 'ner','tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler',  'ner'])]

[Τον, ενημέρωσα, για, τη, καμπάνια, bancassurance, και, δεν, ενδιαφέρεται]

In [507]:
com = {'CON_ROW_ID':[1],'CON_COMMENTS':[txt]}

In [508]:
df = pd.DataFrame(com)

In [509]:
df.head()

,CON_ROW_ID,CON_COMMENTS
0,1,Τον ενημέρωσα για τη καμπάνια bancassurance κα...


In [510]:
df = cleanComments(df)

In [511]:
df.head()

,CON_ROW_ID,CON_COMMENTS,tokenized,variable
0,1,Τον ενημέρωσα για τη καμπάνια bancassurance κα...,ενημερωσα καμπανια bancassurance δενθελειδενεν...,"[ενημερωσα, καμπανια, bancassurance, δενθελειδ..."


In [512]:
df_f = getTokens(df)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [513]:
df_f.head(10)

,CON_ROW_ID,variable
0,1,ενημερωσα
0,1,καμπανια
0,1,bancassurance
0,1,δενθελειδενενδιαφερεται


In [514]:
    tokenCount = df_f['variable'].value_counts().to_dict()
    

In [515]:
df_f.head(10)

,CON_ROW_ID,variable
0,1,ενημερωσα
0,1,καμπανια
0,1,bancassurance
0,1,δενθελειδενενδιαφερεται


In [516]:
df_f['value'] = df_f['variable'].map(tokenCount)


In [517]:
df_f.head(10)

,CON_ROW_ID,variable,value
0,1,ενημερωσα,1
0,1,καμπανια,1
0,1,bancassurance,1
0,1,δενθελειδενενδιαφερεται,1


In [518]:
df_f=df_f[df_f['value']>0] 

In [519]:
df_f.head(10)

,CON_ROW_ID,variable,value
0,1,ενημερωσα,1
0,1,καμπανια,1
0,1,bancassurance,1
0,1,δενθελειδενενδιαφερεται,1


In [520]:
df_f.head(10)

,CON_ROW_ID,variable,value
0,1,ενημερωσα,1
0,1,καμπανια,1
0,1,bancassurance,1
0,1,δενθελειδενενδιαφερεται,1


In [521]:
corDict = dict(pd.read_excel("corTokens.xls").to_dict("split")['data'])

In [522]:
df_f['token'] = df_f['variable'].apply(lambda x : correct(x,corDict))

In [523]:
df_f.head(10)

,CON_ROW_ID,variable,value,token
0,1,ενημερωσα,1,ενημερωση
0,1,καμπανια,1,καμπανια
0,1,bancassurance,1,rmv
0,1,δενθελειδενενδιαφερεται,1,δεν θελει/δεν ενδιαφερεται


In [524]:
df_f = df_f[df_f['token'] !='rmv']

In [525]:
df_f.head(10)

,CON_ROW_ID,variable,value,token
0,1,ενημερωσα,1,ενημερωση
0,1,καμπανια,1,καμπανια
0,1,δενθελειδενενδιαφερεται,1,δεν θελει/δεν ενδιαφερεται


In [526]:
df_f = df_f[df_f['token'].str.len() >1]

In [527]:
df_f.head(10)

,CON_ROW_ID,variable,value,token
0,1,ενημερωσα,1,ενημερωση
0,1,καμπανια,1,καμπανια
0,1,δενθελειδενενδιαφερεται,1,δεν θελει/δεν ενδιαφερεται


In [528]:
df_f = df_f.fillna('N/A')

In [529]:
df_f.head(10)

,CON_ROW_ID,variable,value,token
0,1,ενημερωσα,1,ενημερωση
0,1,καμπανια,1,καμπανια
0,1,δενθελειδενενδιαφερεται,1,δεν θελει/δεν ενδιαφερεται


In [530]:
df_f = df_f[['CON_ROW_ID','token']].drop_duplicates()

In [531]:
df_f.head(10)

,CON_ROW_ID,token
0,1,ενημερωση
0,1,καμπανια
0,1,δεν θελει/δεν ενδιαφερεται
